# **1. Initialise the Colab session**




---







## **1.1. Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelator: GPU** *(Graphics processing unit)*


In [ ]:
#@markdown ##Run this cell to check if you have GPU access

import tensorflow as tf
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.')
  print('Did you change your runtime ?')
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')

else:
  print('You have GPU access')
  !nvidia-smi

## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab



# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

# **2. Install StarDist, Cellpose, Pix2pix and dependencies**
---


In [ ]:

#@markdown ##StarDist, Cellpose, Pix2pix, and dependencies


# Install packages which are not included in Google Colab

#!pip uninstall -y -q yellowbrick
#!pip install "opencv-python-headless<4.3"

#!pip install pyopencl

!pip install -q tifffile # contains tools to operate tiff-files
!pip install -q folium==0.2.1
!pip install -q imgaug==0.2.5

!pip install -q PTable # Nice tables
!pip install -q zarr
!pip install -q imagecodecs

!pip install -q wget
!pip install -q memory_profiler
!pip install -q fpdf2


!pip install -q csbdeep  # contains tools for restoration of fluorescence microcopy images (Content-aware Image Restoration, CARE). It uses Keras and Tensorflow.
!pip install -q stardist # contains tools to operate STARDIST.
!pip install -q gputools # improves STARDIST performances



%load_ext memory_profiler

!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')
!pip install -r requirements.txt
!pip install fpdf

#Force session restart
#exit(0)





In [ ]:
#@markdown ##Load the dependencies and functions
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
np.random.seed(42)
import tensorflow
print(tensorflow.__version__)
print("Tensorflow enabled.")

import imagecodecs

# ------- Variable specific to Stardist -------
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available, relabel_image_stardist, random_label_cmap,  relabel_image_stardist, _draw_polygons, export_imagej_rois
from stardist.models import Config2D, StarDist2D, StarDistData2D # import objects
from stardist.matching import matching_dataset
from csbdeep.utils import Path, normalize, download_and_extract_zip_file, plot_history # for loss plot
from csbdeep.io import save_tiff_imagej_compatible
%matplotlib inline

lbl_cmap = random_label_cmap()

import glob
import os.path
import os
import re
import cv2
from PIL import Image
import zarr
from zipfile import ZIP_DEFLATED
from csbdeep.data import Normalizer, normalize_mi_ma
import imagecodecs

class MyNormalizer(Normalizer):
    def __init__(self, mi, ma):
            self.mi, self.ma = mi, ma
    def before(self, x, axes):
        return normalize_mi_ma(x, self.mi, self.ma, dtype=np.float32)
    def after(*args, **kwargs):
        assert False
    @property
    def do_after(self):
        return False

# ------- Common variable to all ZeroCostDL4Mic notebooks -------
import numpy as np
from matplotlib import pyplot as plt
import urllib
import shutil
from tifffile import imread, imsave
import time
import wget
from pathlib import Path
import pandas as pd
import csv
#from skimage import io
from skimage import io, color, filters, morphology, segmentation
from sklearn.linear_model import LinearRegression
from skimage.util import img_as_uint
import matplotlib as mpl
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr
from astropy.visualization import simple_norm
from skimage import img_as_float32, img_as_ubyte, img_as_float
from skimage.util import img_as_ubyte
from tqdm import tqdm
from fpdf import FPDF, HTMLMixin
from datetime import datetime
from pip._internal.operations.freeze import freeze
import subprocess

# For sliders and dropdown menu and progress bar
from ipywidgets import interact
import ipywidgets as widgets

# Colors for the warning messages
class bcolors:
  WARNING = '\033[31m'
W  = '\033[0m'  # white (normal)
R  = '\033[31m' # red

import imageio
from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms
import os.path


# ------- Common variable to all ZeroCostDL4Mic notebooks -------
import numpy as np
from matplotlib import pyplot as plt
import urllib
import os, random
import shutil
import zipfile
from tifffile import imread, imsave
import sys
from pathlib import Path
import pandas as pd
import csv
from glob import glob
from scipy import signal
from scipy import ndimage
from skimage import io
from sklearn.linear_model import LinearRegression
from skimage.util import img_as_uint
import matplotlib as mpl
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr
from astropy.visualization import simple_norm
from skimage import img_as_float32
from skimage.util import img_as_ubyte
from tqdm import tqdm
from fpdf import FPDF, HTMLMixin
from datetime import datetime
import subprocess
from pip._internal.operations.freeze import freeze
from skimage import data
from skimage import filters

# ------- Variable specific to Cellpose -------

from urllib.parse import urlparse
%matplotlib inline
from skimage.util import img_as_ubyte
import cv2
from ipywidgets import interact, interact_manual
from zipfile import ZIP_DEFLATED


# For sliders and dropdown menu and progress bar
from ipywidgets import interact
import ipywidgets as widgets


# ------- Variable specific to watershed -------
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max


# Download the StarDist model

pretrained_model_name = "2D_versatile_fluo"
pretrained_model_path = "/content/"+pretrained_model_name

if os.path.exists(pretrained_model_path):
  shutil.rmtree(pretrained_model_path)
os.makedirs(pretrained_model_path)

wget.download("https://cloud.mpi-cbg.de/index.php/s/1k5Zcy7PpFWRb0Q/download?path=/versatile&files=2D_versatile_fluo.zip", pretrained_model_path)

with zipfile.ZipFile(pretrained_model_path+"/2D_versatile_fluo.zip", 'r') as zip_ref:
  zip_ref.extractall(pretrained_model_path)

h5_file_path = os.path.join(pretrained_model_path, "weights_best.h5")


# StarDist prediction function

def stardist(video, Saving_folder, model_name, model_path):

  # normalize channels independently
  axis_norm = (0,1)
  model = StarDist2D(None, name = model_name, basedir = model_path)

  Number_of_nuclei_list = []
  Number_of_frame_list = []
  video = normalize(video, 1,99.8, axis=(0,)+tuple(1+np.array(axis_norm)))

  n_timepoint = video.shape[0]
  prediction_stack = np.zeros((n_timepoint, video.shape[1], video.shape[2]), dtype=np.float32)

# Analyse each time points one after the other

  for t in range(n_timepoint):
    img_t = video[t]
    labels, polygons = model.predict_instances(img_t)
    prediction_stack[t] = labels
    Nuclei_array = polygons['coord']
    Nuclei_array2 = [str(t), Nuclei_array.shape[0]]
    Number_of_nuclei_list.append(Nuclei_array2)
    Number_of_frame_list.append(t)

  prediction_stack = img_as_float32(prediction_stack, force_copy=False)

# Export a csv file containing the number of nuclei detected at each frame
  my_df = pd.DataFrame(Number_of_nuclei_list)
  my_df.to_csv(Saving_folder+'/'+str(short_name[0])+'_Nuclei_number.csv', index=False, header=False)
  os.chdir(Saving_folder)
  imsave(str(short_name[0])+".tif", prediction_stack, compression ='zlib')


  # Object detected vs frame number
  plt.figure(figsize=(20,5))
  my_df.plot()
  plt.title('Number of objects vs frame number')
  plt.ylabel('Number of detected objects')
  plt.xlabel('Frame number')
  plt.legend()
  plt.savefig(Saving_folder+'/'+str(short_name[0])+'_Object_detected_vs_frame_number.png',bbox_inches='tight',pad_inches=0)
  plt.show()


# Pix2pix prepare the data function

def pix2pix_prepare_data(timelapse):
  n_timepoint = timelapse.shape[0]
  for t in range(n_timepoint):
    img_t = timelapse[t]
    img_t = exposure.equalize_adapthist(img_t, clip_limit=0.03)
    img_t = 255 * img_t # Now scale by 255
    img_t = img_t.astype(np.uint8)
    cv2.imwrite(testA_Folder+"/"+short_name[0]+"_"+str(t)+".png", img_t)
    cv2.imwrite(testB_Folder+"/"+short_name[0]+"_"+str(t)+".png", img_t)

# Here we create a merged A / A image for the prediction
  os.chdir("/content")
  !python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A "$imageA_folder" --fold_B "$imageB_folder" --fold_AB "$imageAB_folder"

# Pix2pix functions
def pix2pix_WF(imageAB_folder, Prediction_model_name, Prediction_model_path, Image_min_dim, patch_size, checkpoint, saving_folder):

  os.chdir("/content")
  !python pytorch-CycleGAN-and-pix2pix/test.py --dataroot "$imageAB_folder" --name "$Prediction_model_name" --model pix2pix --no_dropout --preprocess resize_and_crop --load_size $load_size --crop_size $crop_size --results_dir "$saving_folder" --checkpoints_dir "$Prediction_model_path" --num_test $n_timepoint --epoch $checkpoint --input_nc "1" --output_nc "1" --dataset_mode "aligned"

  Checkpoint_name = "test_"+str(checkpoint)

  Prediction_results_folder = saving_folder+"/"+Prediction_model_name+"/"+Checkpoint_name+"/images"

  Prediction_results_images = os.listdir(Prediction_results_folder)

  for f in Prediction_results_images:
    if (f.endswith("_fake_B.png")):
      shutil.copyfile(Prediction_results_folder+"/"+f,saving_folder+"/"+f)

  shutil.rmtree(saving_folder+"/"+Prediction_model_name)


def generate_stack(input_folder, Saving_folder):
# Here we check the image dimensions
  random_choice = random.choice(os.listdir(input_folder))
  x = io.imread(input_folder+"/"+random_choice)
  Image_Y = x.shape[0]
  Image_X = x.shape[1]

#Here we load all the images as a stack to generate the projection
  n_images = len([name for name in os.listdir(input_folder)])
  image_stack = np.zeros((n_images, Image_Y, Image_X))
  Images_name = os.listdir(input_folder)
  for t in range(n_images):
    Image_t = io.imread(input_folder+"/"+Images_name[t], as_gray= True)
    image_stack[t] = Image_t
  os.chdir(Saving_folder)
  imsave(str(short_name[0])+"_stack.tif", image_stack)

def load_images_from_folder(folder, Saving_folder):
    images = []
    # Find all the files in the folder
    for filename in os.listdir(folder):
        # Only consider files that end with .jpg (or modify to suit your file types)
        if filename.endswith(".png"):
            img = io.imread(os.path.join(folder, filename), as_gray= True)
            if img is not None:
                # Remove the common prefix from filename
                filename = filename.replace(short_name[0], "")
                # Get the number from the file name using regex
                number = re.search(r'(\d+)', filename)
                if number is not None:
                    images.append((int(number.group()), img))  # Use the matched number for sorting
    # Sort images according to the number in their name
    images.sort(key=lambda x: x[0])
    # Take only image data, discarding the numbers
    images = [img for num, img in images]
    # Stack images
    stacked_images = np.stack(images)
    os.chdir(Saving_folder)
    imsave(str(short_name[0])+"_pix2pixstack.tif", stacked_images)
    del stacked_images

def combinelabels(Result_folder):
  StarDist_BF = io.imread(Saving_folder_StarDist_cancer_cells_BF +"/"+short_name[0] +".tif")
  StarDist_Both = io.imread(Saving_folder_StarDist_cancer_cells_both +"/"+short_name[0] +".tif")
  StarDist_pix2pix = io.imread(Saving_folder_StarDist_cancer_cells_pix2pix +"/"+short_name[0] +".tif")

  StarDist_BF[StarDist_BF>0]=1
  StarDist_Both[StarDist_Both>0]=1
  StarDist_pix2pix[StarDist_pix2pix>0]=1

  label = StarDist_BF+StarDist_Both+StarDist_pix2pix
  os.chdir(Result_folder)
  imsave(str(short_name[0])+"combined_labels.tif", label, compression ='zlib')
  label[label>0]=1
  imsave(str(short_name[0])+"combined_mask.tif", label, compression ='zlib')
  return label

def final_seg(Results_folder, save_Final):
    video_path = os.path.join(Results_folder, short_name[0], f"{short_name[0]}combined_mask.tif")
    video = io.imread(video_path)
    video = (255 * video).astype(np.uint8)

    n_timepoint = video.shape[0]
    prediction_stack = np.zeros((n_timepoint, video.shape[1], video.shape[2]), dtype=np.float32)

    # Analyze each time points one after the other
    for t in range(n_timepoint):
        image = video[t]
        distance = ndi.distance_transform_edt(image)
        coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)
        mask = np.zeros(distance.shape, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndi.label(mask)
        prediction_stack[t] = watershed(-distance, markers, mask=image)

    prediction_stack = img_as_float32(prediction_stack, force_copy=False)

    os.chdir(save_Final)
    imsave(str(short_name[0])+"final_mask.tif", prediction_stack, compression ='zlib')

    fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow(video[1], cmap=plt.cm.gray)
    ax[0].set_title('Overlapping objects')
    ax[1].imshow(-distance, cmap=plt.cm.gray)
    ax[1].set_title('Distances')
    ax[2].imshow(prediction_stack[1], cmap=plt.cm.nipy_spectral)
    ax[2].set_title('Separated objects')

    for a in ax:
        a.set_axis_off()

    fig.tight_layout()
    plt.show()

def plotresults(Result_folder):
  lbl_cmap = random_label_cmap()

  StarDist_BF = io.imread(Saving_folder_StarDist_cancer_cells_BF +"/"+short_name[0] +".tif")
  StarDist_Both = io.imread(Saving_folder_StarDist_cancer_cells_both +"/"+short_name[0] +".tif")
  StarDist_pix2pix = io.imread(Saving_folder_StarDist_cancer_cells_pix2pix +"/"+short_name[0] +".tif")

  f=plt.figure(figsize=(16,8))
  plt.subplot(2,3,1)
  plt.imshow(timelapse_WF[0], cmap="gray", interpolation='nearest')
  plt.title('BF')
  plt.axis('off');
  plt.subplot(2,3,2)
  plt.imshow(timelapse_both[0], cmap="gray", interpolation='nearest')
  plt.title('Both Channel')
  plt.axis('off');
  plt.subplot(2,3,3)
  plt.imshow(timelapse_pix2pix[0],cmap="gray", interpolation='nearest')
  plt.title('Pix2Pix predictions')
  plt.axis('off');
  plt.subplot(2,3,4)
  plt.imshow(StarDist_BF[0], cmap=lbl_cmap, interpolation='nearest')
  plt.title('StarDist_BF')
  plt.axis('off');
  plt.subplot(2,3,5)
  plt.imshow(StarDist_Both[0], cmap=lbl_cmap, interpolation='nearest')
  plt.title('StarDist_Both')
  plt.axis('off');
  plt.subplot(2,3,6)
  plt.imshow(StarDist_pix2pix[0], cmap=lbl_cmap, interpolation='nearest')
  plt.title('StarDist_pix2pix')
  plt.axis('off');
  plt.savefig(Result_folder+'/'+str(short_name[0])+'_results.png',bbox_inches='tight',pad_inches=0)
  plt.show()

print('----------------------------')
print("Libraries installed")





# **6. Select your parameters and paths**
---

In [ ]:
latest = "latest"

#@markdown #Provide the path to your dataset:

BF_folder = "/content/gdrive/Shareddrives/Kurppa_colab/Data_to_process/6.4Results_stagreg_1024/BF-corrected" #@param {type:"string"}
Both_folder = "/content/gdrive/Shareddrives/Kurppa_colab/Data_to_process/6.4Results_stagreg_1024/both-corrected" #@param {type:"string"}
Results_folder = "/content/gdrive/Shareddrives/Kurppa_colab/Data_to_process/240523_results" #@param {type:"string"}

#@markdown #StarDist models locations:

StarDist_model_cancer_cells_BF = "/content/gdrive/Shareddrives/Kurppa_colab/Stardist_BF_round_cells/3-models/Stardist_BF_round_cell_200aug" #@param {type:"string"}
StarDist_model_both = "/content/gdrive/Shareddrives/Kurppa_colab/StarDist-fluorescence_training/3-model/Kurppa-Stardist-200e+aug" #@param {type:"string"}


#@markdown #Pix2pix models locations:


Pix2pix_model_BF_to_both = "/content/gdrive/Shareddrives/Kurppa_colab/Pix2pix/3_model/pix2pix-kurppa-400-1740pairs-3" #@param {type:"string"}

Pix2pix_model_Nuclei_checkpoint = 50#@param {type:"raw"}


# Here we check that the StarDist BF models exist

StarDist_cancer_cells_BF_model_name = os.path.basename(StarDist_model_cancer_cells_BF)
StarDist_cancer_cells_BF_model_path = os.path.dirname(StarDist_model_cancer_cells_BF)

StarDist_cancer_cells_BF_full_model_path = StarDist_cancer_cells_BF_model_path+'/'+StarDist_cancer_cells_BF_model_name+'/'

# Here we check that the pix2pix models exist
if os.path.exists(StarDist_cancer_cells_BF_full_model_path):
  print("The "+StarDist_cancer_cells_BF_model_name+" network will be used to detect round cancer cells in BF image.")
else:
  print(bcolors.WARNING+'!! WARNING: The chosen StarDist model for detecting the cancer cells does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')

StarDist_model_both_name = os.path.basename(StarDist_model_both)
StarDist_model_both_path = os.path.dirname(StarDist_model_both)

StarDist_model_both_full_model_path = StarDist_model_both_path+'/'+StarDist_model_both_name+'/'

# Here we check that the pix2pix models exist
if os.path.exists(StarDist_model_both_full_model_path):
  print("The "+StarDist_model_both_name+" network will be used to detect cancer cells in the both image.")
else:
  StarDist_model_cells_name = "fluo"

#Here we find the loaded model name and parent path
Pix2pix_model_Nuclei_model_name = os.path.basename(Pix2pix_model_BF_to_both)
Pix2pix_model_Nuclei_model_path = os.path.dirname(Pix2pix_model_BF_to_both)

#here we check if the pix2pix model exists
Pix2pix_model_Nuclei_full_model_path = Pix2pix_model_Nuclei_model_path+'/'+Pix2pix_model_Nuclei_model_name+'/'

if os.path.exists(Pix2pix_model_Nuclei_full_model_path):
  print("The "+Pix2pix_model_Nuclei_model_name+" network will be used to predict fluo image from bf image.")
else:
  W  = '\033[0m'  # white (normal)
  R  = '\033[31m' # red
  print(R+'!! WARNING: The chosen model does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')



In [ ]:
#@markdown #Let's go

from skimage import io, img_as_float32
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
import numpy as np
import matplotlib.pyplot as plt
import os

#create path for the final sequences
save_Final = Results_folder+"/"+ "final_mask"

if os.path.exists(save_Final):
  shutil.rmtree(save_Final)
os.makedirs(save_Final)

#Here we perform the predictions

for image in os.listdir(BF_folder):

  print("Performing prediction on: "+image)

  #here we open the two required dataset

  timelapse_WF = io.imread(BF_folder+"/"+image)
  timelapse_both = io.imread(Both_folder+"/"+image)


  #here we get info on the images
  short_name = os.path.splitext(image)
  Image_Y = timelapse_WF.shape[1]
  Image_X = timelapse_WF.shape[2]
  n_timepoint = timelapse_WF.shape[0]

  load_size = 1024 #change here to change the output for pix2pix
  #Image_min_dim = 992 #change here to change the output for pix2pix

  crop_size = 1024
  #patch_size = 992

# here we create the folder where to save the data

  Saving_folder = Results_folder+"/"+short_name[0]

  if os.path.exists(Saving_folder):
    shutil.rmtree(Saving_folder)
  os.makedirs(Saving_folder)

 # Here we actually start analysing the data

  #pix2pix predictions.

  #here we prepare the folder for pix2pix

  Saving_path_pix2pix = "/content/pix2pix"
  if os.path.exists(Saving_path_pix2pix):
    shutil.rmtree(Saving_path_pix2pix)
  os.makedirs(Saving_path_pix2pix)


  imageA_folder = Saving_path_pix2pix+"/Raw/A"
  os.makedirs(imageA_folder)

  imageB_folder = Saving_path_pix2pix+"/Raw/B"
  os.makedirs(imageB_folder)

  imageAB_folder = Saving_path_pix2pix+"/Raw/AB"
  os.makedirs(imageAB_folder)

  testAB_Folder = Saving_path_pix2pix+"/Raw/AB/test"
  os.makedirs(testAB_Folder)

  testA_Folder = Saving_path_pix2pix+"/Raw/A/test"
  os.makedirs(testA_Folder)

  testB_Folder = Saving_path_pix2pix+"/Raw/B/test"
  os.makedirs(testB_Folder)

  Saving_path_pix2pix_WF_DAPI = Saving_path_pix2pix +"/predictions"
  os.makedirs(Saving_path_pix2pix_WF_DAPI)

  #pix2pix prepare the data
  pix2pix_prepare_data(timelapse_WF)

  #pix2pix nuclei
  pix2pix_WF(imageAB_folder, Pix2pix_model_Nuclei_model_name, Pix2pix_model_Nuclei_model_path, load_size, crop_size, Pix2pix_model_Nuclei_checkpoint, Saving_path_pix2pix_WF_DAPI)

  #create a stack from pix2pix prediction.

  #generate_stack(Saving_path_pix2pix_WF_DAPI, Saving_path_pix2pix)
  load_images_from_folder(Saving_path_pix2pix_WF_DAPI, Saving_folder)

  #StarDist predictions.

  #Here we identify the cancer cells from the BF data

  Saving_folder_StarDist_cancer_cells_BF = Saving_folder+"/"+"Cancer_cells_BF"
  if os.path.exists(Saving_folder_StarDist_cancer_cells_BF):
    shutil.rmtree(Saving_folder_StarDist_cancer_cells_BF)
  os.makedirs(Saving_folder_StarDist_cancer_cells_BF)

  stardist(timelapse_WF, Saving_folder_StarDist_cancer_cells_BF, StarDist_cancer_cells_BF_model_name, StarDist_cancer_cells_BF_model_path)


  #StarDist predictions. Here we identify the cancer cells from the both channel

  Saving_folder_StarDist_cancer_cells_both = Saving_folder+"/"+"Cancer_cells_both"
  if os.path.exists(Saving_folder_StarDist_cancer_cells_both):
    shutil.rmtree(Saving_folder_StarDist_cancer_cells_both)
  os.makedirs(Saving_folder_StarDist_cancer_cells_both)

  stardist(timelapse_both, Saving_folder_StarDist_cancer_cells_both, StarDist_model_both_name, StarDist_model_both_path)

  #StarDist predictions. Here we identify the cancer cells from the pix2pix prediction

  Saving_folder_StarDist_cancer_cells_pix2pix = Saving_folder+"/"+"Cancer_cells_pix2pix"
  if os.path.exists(Saving_folder_StarDist_cancer_cells_pix2pix):
    shutil.rmtree(Saving_folder_StarDist_cancer_cells_pix2pix)
  os.makedirs(Saving_folder_StarDist_cancer_cells_pix2pix)

  timelapse_pix2pix = io.imread(Saving_folder+"/"+short_name[0]+"_pix2pixstack.tif")

  stardist(timelapse_pix2pix, Saving_folder_StarDist_cancer_cells_pix2pix, StarDist_model_both_name, StarDist_model_both_path)

  # make a figure
  plotresults(Saving_folder)

  # combine the labels

  combinelabels(Saving_folder)

  # watershed
  #final_seg(Results_folder,save_Final)

  # Final StarDist

  video_path = os.path.join(Results_folder, short_name[0], f"{short_name[0]}combined_mask.tif")
  video = io.imread(video_path)

  stardist(video, save_Final, pretrained_model_name, "/content")





